In [1]:
from datasets import load_dataset, load_from_disk
import os
import sys
print(os.getcwd())
print(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.getcwd()))
# print(sys.path)
from src.utils import translate_to_phonetic
from src.train_tokenizer import load_tokenizer
from src.bert_wikitext_train import preprocess_dataset

/home/toure215/BERT_phonetic/test
/home/toure215/BERT_phonetic


In [2]:
bookcorpus = load_from_disk("/home/toure215/BERT_phonetic/DATASETS/phonetic_bookcorpus")

Loading dataset from disk:   0%|          | 0/47 [00:00<?, ?it/s]

In [3]:
bookcorpus

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 150851682
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 300659
    })
})

In [4]:
print(bookcorpus["train"][0])

{'text': 'wi Ol tr\\=nd t@ si dIl@n st{ndIN {t D@ bAt@m Vv D@ stEr\\z In nVTIN bVt p@dZAm@ bAt@mz '}


In [5]:
text_lengths = bookcorpus['train'].map(lambda x: {"text_length": len(x["text"].split(" "))}, num_proc=16, remove_columns=["text"])
total_length = sum(text_lengths["text_length"])
mean = total_length / len(bookcorpus["train"])
mean

Map (num_proc=16):   0%|          | 0/150851682 [00:00<?, ? examples/s]

30.14613023671821

In [8]:
print(text_lengths[:10])
print(bookcorpus["train"][6])
print(max(text_lengths["text_length"]))

{'text_length': [19, 40, 47, 5, 50, 48, 12, 19, 29, 48]}
{'text': 'lEt  Es nAt dZVmp t@ k@nkluZ@nz bIkOz Vv pr\\=s@n@l baj@s '}
50


In [9]:
dataset = load_dataset("glue", "mrpc")

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [11]:
dataset = dataset.remove_columns(["sentence1", "idx", "label"]).rename_column("sentence2", "text")
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 408
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1725
    })
})

In [20]:
from functools import lru_cache
import re
import epitran

epi = epitran.Epitran("eng-Latn")


@lru_cache(maxsize=None)
def cached_xsampa(word):
    return "".join(epi.xsampa_list(word))


def xsampa(sentences):
    return [
        " ".join(map(cached_xsampa, re.findall(r"\w+|[^\s\w]+", sentence)))
        for sentence in sentences
    ]


def translate_to_phonetic(example):
    sentence = xsampa(example["text"])
    return {"text": sentence}

In [21]:
phonetic_dataset = dataset.map(translate_to_phonetic, num_proc=16, batched=True)

Map (num_proc=16):   0%|          | 0/3668 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/408 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/1725 [00:00<?, ? examples/s]